Получение датесета

Гороскопы взяты с этого сайта https://raw.githubusercontent.com/dkagramanyan/astro/main/horoscopes.csv

Просмотр статистики и объема датасета

Датасет состоит примерно из 70к предсказаний. Для обучения выбрано по 50 позитивных и негативных предсказаний. Каждое содержит до 150 символов текста

In [ ]:
!pip install transformers
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 37.0 MB/s 
     |████████████████████████████████| 7.6 MB 48.2 MB/s 
     |████████████████████████████████| 182 kB 65.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 29.8 MB/s 
     |████████████████████████████████| 115 kB 67.1 MB/s 
     |████████████████████████████████| 212 kB 54.4 MB/s 
     |████████████████████████████████| 127 kB 66.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# использовать из низа модель
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [ ]:
#Предобработка данных
from datasets import load_dataset, Dataset
dataset = load_dataset("text", data_files={"train": "./positive.txt"}, sample_by="document")
horoscope_datasets = []
for index, item in enumerate(dataset['train'][0]['text'].split('\n\n')):
    if (index > 50): # длинна датасета
        break
    horoscope_datasets.append({"text":item})

dataset = Dataset.from_list(horoscope_datasets, split="train+test")
dataset[0]
dataset = dataset.map(lambda examples: tokenizer(examples["text"]))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-2e92891f3be0e5b3/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?ex/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Выдвижение гипотез


Модель будет генерировать гороскопы похожие на те, что в датасете и дополнять их

Выбор модели

Для генерации текста выбрана gpt3: rugpt3small_based_on_gpt2

Обучение модели

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuner",
    learning_rate= 2e-3,
    per_device_train_batch_size = 1, # если позволяет память - можно сделать больше
    per_device_eval_batch_size=1,
    num_train_epochs=500, # сколько будет пройден датасет полностью
    weight_decay=0.01,
    save_total_limit = 1
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=2e-3), None) # Optimizer and lr scheduler
)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})


Assigning [PAD] to the pad_token key of the tokenizer
Adding [PAD] to the vocabulary


1

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 49
  Num Epochs = 500
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 24500
  Number of trainable parameters = 355871744


Step,Training Loss
500,3.782200
1000,2.418500
1500,2.068800
2000,1.901300
2500,1.988100
3000,1.993700
3500,2.068900
4000,2.149600
4500,2.256400
5000,2.071800


Saving model checkpoint to ./finetuner/checkpoint-500
Configuration saved in ./finetuner/checkpoint-500/config.json
Model weights saved in ./finetuner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./finetuner/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./finetuner/checkpoint-500/special_tokens_map.json
added tokens file saved in ./finetuner/checkpoint-500/added_tokens.json
Saving model checkpoint to ./finetuner/checkpoint-1000
Configuration saved in ./finetuner/checkpoint-1000/config.json
Model weights saved in ./finetuner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./finetuner/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./finetuner/checkpoint-1000/special_tokens_map.json
added tokens file saved in ./finetuner/checkpoint-1000/added_tokens.json
Deleting older checkpoint [finetuner/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./finetuner/checkpoint-1500
Configuration saved in ./

TrainOutput(global_step=24500, training_loss=1.2170135348573023, metrics={'train_runtime': 6685.5152, 'train_samples_per_second': 3.665, 'train_steps_per_second': 3.665, 'total_flos': 1109180614656000.0, 'train_loss': 1.2170135348573023, 'epoch': 500.0})

Расчет метрик

loss полученный при обучении модели равен 0.2677

---



Сохранение модели. 

Модель сохранена в папку checkpoint-24500, для загрузки используется следующий код:

In [ ]:
from transformers import AutoConfig, GPT2Config
label2id= {
    "negative":0,
    "positive":1,
}
id2label= {
    0:"negative",
    1:"positive",
}
config = GPT2Config.from_pretrained("./finetuner/checkpoint-24500", label2id=label2id, id2label=id2label)

loading configuration file ./finetuner/checkpoint-24500/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3medium_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "n_special": 0,
  "output_past": true,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj"

In [ ]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = GPT2Tokenizer.from_pretrained("finetuner/checkpoint-24500")
model = GPT2LMHeadModel.from_pretrained("finetuner/checkpoint-24500",config=config).to(DEVICE) # Загружаем обученную модель из чекпоинтов

loading file vocab.json
loading file merges.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding [PAD] to the vocabulary
loading weights file finetuner/checkpoint-24500/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at finetuner/checkpoint-24500.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


Генерация гороскопа

In [ ]:
text = "[SG]"
input_ids = tokenizer.encode(text, return_tensors="pt").to("cuda:0")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100, pad_token_id=tokenizer.eos_token_id,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



[SG]"Это хороший день для проявления интеллектуальной активности."
[EG]]]][EG]]][EG]]]][EG][EG]][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG][EG]]][EG][EG][EG]]]]


Вывод

В данной ЛР я узнал как подготавливать данные для обучения, а также обучать, и запускать модель.